## IBM Model1
### Podstawowe równanie
Główne równanie machine translation

$$p(\mathrm{f},\mathrm{a}|\mathrm{e}) = \frac{\epsilon}{(l+1)^m} \prod_{j=1}^m t(f_j|e_{a(j)})$$

Gdzie:
- zdanie $\mathrm{f}$ to zdanie zagraniczne $\mathrm{f}=f_1...f_m$,
- zdanie wejsciowe to $\mathrm{e}=e_1...e_m$
- każde słowo zagraniczne $f_i$ zostało wygenerowane ze słowa wejściowego $e_{a(j)}$ poprzez funkcję zrównoleglania $a$, z prawdopodobieństwem $t$
- $\epsilon = P(m|e)$ to stała normalizacyjna


## Algorytm EM
Algorytm EM składa się z dwóch kroków:
1. Krok Expectatio: aplikuje model do dnaych
    - część modelu jest ukryta (w tym wypadku zrównoleglenie)
    - używając modelu przypisujemy prawdopodobieństwo do możliwych wartości
2. Krok Maximization: estymuje model na podstawie danych
    - przyjmuje przypisane wartości jakofakt
    - liczby powiązania (counts) i odpowiadające im wagi (prawdopodobieństwa)
    - estymuje nowy model na postawie policzonych powiązań (counts)
   
Powtarza powyższe kroki aż nie osiągnie zbieżności.

### EM - Krok Expectation
Musimy oszacować (expect) liczbę razy kiedy słowo $e$ łączy się ze słowem $f$ w tłumaczeniu $(\mathrm{f}|\mathrm{e})$. Tą wartość możemy oszacować wzorem:

$$ c(f|e;\mathrm{f};\mathrm{e})=\sum_{\mathrm{a}} P(\mathrm{a}|\mathrm{e},\mathrm{f})\sum_{j=1}^m \delta(f,f_j)\delta(e,e_{a(j)})$$

Gdzie ta druga suma oznacza po prostu liczbę łączy słowo $e$ ze słowem $f$ w zdaniu $\mathrm{a}$.

W jaki sposób wyliczyć $p(\mathrm{a}|\mathrm{e},\mathrm{f})$?
$$p(\mathrm{a}|\mathrm{e},\mathrm{f}) = \frac{ p(\mathrm{f},\mathrm{a}|\mathrm{e})}{p(\mathrm{f}|\mathrm{e})}$$

A w jaki sposób obliczyć samo $p(\mathrm{f},\mathrm{a}|\mathrm{e})$? Jest to wzór zdefiniowany przez Model 1.

Jak wyliczyć $p(\mathrm{f}|\mathrm{e})$?
![alt text](translation1.png "Sposób obliczenia prawdopodobieństwa tłumaczenia zdania f przez zdanie e")
Sposób obliczenia prawdopodobieństwa tłumaczenia zdania f przez zdanie e


### EM - Krok Maximization
Pierwszym zadaniem w tym kroku jest policzenie powiązań (counts).
Dowód z par zdań $\mathrm{e}, \mathrm{f}$, że słowo $f$ jest tłumaczeniem słowa $f$ obliczany jest w następujący sposób:

$$c(f|e;\mathrm{f},\mathrm{f})=\sum_a p(\mathrm{a}|\mathrm{e}, \mathrm{f})\sum_{j=1}^m \delta(f,f_j)\delta(e,e_{a(j)})$$

A używając uproszenia możemy zapisać to jako:

$$c(f|e;\mathrm{f},\mathrm{f})=\frac{t(f|e)}{\sum_{i=0}^lt(f|e_i)} \sum_{j=1}^m\delta(f,f_j)\delta(e,e_i)$$

Po obliczeniu tych wartości możemy oszacować model:

$$t(f|e;\mathrm{e},\mathrm{f})=\frac{\sum_{(\mathrm{e},\mathrm{f})}c(f|e;\mathrm{e},\mathrm{f})}{\sum_{f_i}\sum_{(\mathrm{e},\mathrm{f})}c(f_i|e;\mathrm{e},\mathrm{f})}$$

In [41]:
flatten = lambda l : [item for sublist in l for item in sublist]

In [42]:
e_s_set = map(str.split, ["ja mam domek", "ja domek mam", "domek", "ja"])
f_s_set = map(str.split, ["i have house", "i have house", "house", "i"])

In [43]:
def EM(f_s_set, e_s_set):
    f_set = set(flatten(f_s_set))
    e_set = set(flatten(e_s_set))
    
    #initialize t(f|e) uniformly
    t = {}
    for f in f_set:
        for e in e_set:
            t[(f, e)] = 1.0/len(e_set)

    improvement = 1
    while improvement > 0.1:
        count = {(f, e):0 for f, e in t.keys()}
        total = {e:0 for e in e_set}

        for f_s, e_s in zip(f_s_set, e_s_set):  # for each pair of sentences
            for f in set(f_s):  # for each word
                n_f = f_s.count(f)  # count of this word in this sentence
                total_s = 0
                for e in set(e_s):
                    total_s += t[(f, e)] * n_f

                for e in set(e_s):
                    n_e = e_s.count(e)
                    count[(f, e)] += t[(f, e)] * n_f * n_e / total_s
                    total[e] += t[(f, e)] * n_f * n_e / total_s

                # tu liczymy prawdopodobieństwo warunkowe dla p(f|e)
                # total_s to mianowanik - suma wag * liczba słów
                # count przechowuje liczniki
                # total przechowuje prawdopodobieństwo słowa e

        improvement = 0
        for f, e in t.keys():
            if e in total:
                if total[e] > 0:
                    new_value = count[(f, e)] / total[e]
                    improvement += abs(t[(f, e)] - new_value)
                    t[(f, e)] = new_value

    return t


In [44]:
t = EM(f_s_set, e_s_set)

for pair, score in sorted(t.items(), key=lambda x:x[1], reverse=True):
    print pair, score

('house', 'domek') 0.971125382219
('i', 'ja') 0.971125382219
('have', 'mam') 0.846928773971
('house', 'mam') 0.0765356130144
('i', 'mam') 0.0765356130144
('have', 'domek') 0.0264815243342
('have', 'ja') 0.0264815243342
('i', 'domek') 0.0023930934463
('house', 'ja') 0.0023930934463
